# 호텔 챗봇

이 노트북에서는 호텔 챗봇 문제를 다룹니다. 어떻게 하면 호텔 프론트 데스크에서 더 나은 고객 서비스를 제공할 수 있을까요? 간단한 방법 중 하나는 호텔에서 경험할 수 있는 일들에 관한 간단한 질문에 대답할 수 있는 FAQ 채팅 로봇을 갖는 것입니다. 챗봇을 갖는 것에는 많은 장점이 있습니다.

1. 호텔의 매력을 높이고 정보 처리량을 늘립니다.
2. 호텔에 대한 질문을 데이터 테이블 형식으로 수집할 수 있는 방법을 만듭니다.

이 노트북에서는 두 가지 모델을 살펴볼 것입니다. 코사인 유사성 및 doc2vec 모델입니다.


## 1.챗봇에 지식 기반 추가

챗봇의 대화 능력은 사용할 수 있는 데이터에 의해 정의됩니다. Ques.txt 파일과 ans.txt 파일에서 질문과 답변을 살펴보십시오. 이 챗봇은 기본적으로 질문에 대하여 질문 은행과 코사인 유사성을 확인하여 답을 찾으려고 노력할 것입니다.

### 1.1 라이브러리 가져오기

In [ ]:
import nltk # 텍스트 데이터를 처리
import numpy as np # 말뭉치를 배열로 표현
import random
import operator
import string # 표준 파이썬 문자열을 처리
from sklearn.metrics.pairwise import cosine_similarity # 이를 나중에 사용하여 두 개의 문장이 얼마나 비슷한지를 결정합니다.
from sklearn.feature_extraction.text import TfidfVectorizer # Experience 2에서 단어 가방을 만드는 함수를 만들었던 것을 기억하십니까? 이 함수는 같은 일을 합니다!

### 1.2 데이터 처리
#### 1.2.1 파일 열기

In [ ]:
import os
filepath=os.getcwd()+r'//[Dataset] Module27 (ans).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data_ans=corpus.read()
print (raw_data_ans)

filepath=os.getcwd()+'//[Dataset] Module27(ques).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data=corpus.read()
print (raw_data)

200$ per night is the price for a basic suite.

This establishment was constructed and inaugurated by John S. on the 23rd of September 1965.

Breakfast is served from 7 AM to 10 AM.

The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before. 

The Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.

There are 43 rooms in this hotel including one pent house suite.

We offer 4 types of rooms: basic, mid-level, premium and penthouse.

This hotel is called Vance Hotel.

Yes, room service is available 24 hrs.

To call room service, please dial '0' using the phone in your room.

Yes we have one restaurant currently called 'Rouge'.

There are 12 floors in the hotel.

300$ per night is the standard price for a mid-level suite.

500$ per night is the price for a premium suite.

Yes, we

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer  = WordNetLemmatizer()



# 파일 내용 읽기
with open("[Dataset] Module27(ques).txt", "r") as file:
    questions = file.readlines()

with open("[Dataset] Module27 (ans).txt", "r") as file:
    answers = file.readlines()

# 각 질문과 답변의 앞뒤 공백 제거
questions = [q.strip().lower() for q in questions]
answers = [a.strip().lower() for a in answers]

# 빈 문자열과 None 값을 가진 질문과 답변 제거
questions = [q for q in questions if q]
answers = [a for a in answers if a]

# 질문과 답변 리스트의 길이를 동일하게 맞추기
max_length = max(len(questions), len(answers))
questions += [None] * (max_length - len(questions))
answers += [None] * (max_length - len(answers))

# 데이터 프레임 생성
df = pd.DataFrame({"Question": questions, "Answer": answers})

# 데이터 프레임의 인덱스 재설정
df.reset_index(drop=True, inplace=True)

#질문과 답변 컬럼에 대해 토큰화 수행
df['Question_sent_Tokens'] = df['Question'].apply(lambda x: sent_tokenize(x) if x else [])
df['Answer_sent_Tokens'] = df['Answer'].apply(lambda x: sent_tokenize(x) if x else [])

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

df['Question_word_Tokens'] = df['Question'].apply(lambda x: word_tokenize(x.lower().translate(remove_punct_dict)) if x else [])
df['Answer_word_Tokens'] = df['Answer'].apply(lambda x: word_tokenize(x.lower().translate(remove_punct_dict)) if x else [])


# 각 단어 리스트에 대해 lemmatize 수행
df['Question_stem_Tokens'] = df['Question_word_Tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens] if tokens else [])
df['Answer_stem_Tokens'] = df['Answer_word_Tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens] if tokens else [])
df.head()

,Question,Answer,Question_sent_Tokens,Answer_sent_Tokens,Question_word_Tokens,Answer_word_Tokens,Question_stem_Tokens,Answer_stem_Tokens
0,what is the price of one night stay in basic s...,200$ per night is the price for a basic suite.,[what is the price of one night stay in basic ...,[200$ per night is the price for a basic suite.],"[what, is, the, price, of, one, night, stay, i...","[200, per, night, is, the, price, for, a, basi...","[what, is, the, price, of, one, night, stay, i...","[200, per, night, is, the, price, for, a, basi..."
1,how old is this establishment?,this establishment was constructed and inaugur...,[how old is this establishment?],[this establishment was constructed and inaugu...,"[how, old, is, this, establishment]","[this, establishment, was, constructed, and, i...","[how, old, is, this, establishment]","[this, establishment, wa, constructed, and, in..."
2,what time is breakfast served?,breakfast is served from 7 am to 10 am.,[what time is breakfast served?],[breakfast is served from 7 am to 10 am.],"[what, time, is, breakfast, served]","[breakfast, is, served, from, 7, am, to, 10, am]","[what, time, is, breakfast, served]","[breakfast, is, served, from, 7, am, to, 10, am]"
3,what is the breakfast menu?,the breakfast menu is decided as per the head ...,[what is the breakfast menu?],[the breakfast menu is decided as per the head...,"[what, is, the, breakfast, menu]","[the, breakfast, menu, is, decided, as, per, t...","[what, is, the, breakfast, menu]","[the, breakfast, menu, is, decided, a, per, th..."
4,what is the history behind this hotel or estab...,the vance hotel is a singular establishment de...,[what is the history behind this hotel or esta...,[the vance hotel is a singular establishment d...,"[what, is, the, history, behind, this, hotel, ...","[the, vance, hotel, is, a, singular, establish...","[what, is, the, history, behind, this, hotel, ...","[the, vance, hotel, is, a, singular, establish..."


####  1.2.2 소문자로 변환

모든 텍스트를 먼저 소문자로 변환합니다. 결과가 완료되면 검사해야 합니다.

In [ ]:
raw_data=raw_data.lower() # 소문자로 변환
raw_data_ans=raw_data_ans.lower()

####  세분화, 표제어 추출,  단어 토큰화

모듈 26 코사인 유사성 노트북에서 배운 방법을 참조하여 데이터를 세분화, 토큰화, 표제어 추출로 전처리 진행합니다.

In [ ]:
# 1.2.3 문장 세분화
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')

# question 데이터에 대해
sent_tokens = nltk.sent_tokenize(raw_data)# 문서를 문장 목록으로 변환
print(len(sent_tokens))
print(sent_tokens)


47
['what is the price of one night stay in basic suite?', 'how old is this establishment?', 'what time is breakfast served?', 'what is the breakfast menu?', 'what is the history behind this hotel or establishment?', 'how many rooms are there in this hotel?', 'what are the types of rooms or suites offered by the hotel?', 'what is the name of the hotel?', 'is room service served 24 hours?', 'how do i call for room service?', 'are there any restaurants in the hotel?', 'how many floors are there in the hotel?', 'what is the price of one night stay at the mid-level suite?', 'what is the price of one night stay at the premium suite?', 'do you have tuxedo services?', 'do you have a laundry service?', 'what time do the restaurants open for dinner?', 'what are the near by tourist attractions?', 'is there a spa in the hotel?', 'is there anywhere i can get a massage?', 'what time is the check in?', 'what time is the check out?', 'do you offer handicapped rooms?', 'is parking available at the hot

In [ ]:
len(sent_tokens)

47

#### 1.2.3 문장 세분화

In [ ]:
# answer 데이터에 대해
sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)# 문서를 문장 목록으로 변환
print(sent_tokens_ans)

['200$ per night is the price for a basic suite.', 'this establishment was constructed and inaugurated by john s. on the 23rd of september 1965.\n\nbreakfast is served from 7 am to 10 am.', 'the breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before.', 'the vance hotel is a singular establishment designed and constructed by lindsey vance in 1949; it has hosted many dignitaries and government officials over the years.', 'there are 43 rooms in this hotel including one pent house suite.', 'we offer 4 types of rooms: basic, mid-level, premium and penthouse.', 'this hotel is called vance hotel.', 'yes, room service is available 24 hrs.', "to call room service, please dial '0' using the phone in your room.", "yes we have one restaurant currently called 'rouge'.", 'there are 12 floors in the hotel.', '300$ per night is the standard price for a mid-level suite.', '500$ per night is the price fo

In [ ]:
len(sent_tokens_ans)

46

In [ ]:
min_len = min(len(sent_tokens), len(sent_tokens_ans))
res = {sent_tokens[i]: sent_tokens_ans[i] for i in range(min_len)}

print(res)

{'what is the price of one night stay in basic suite?': '200$ per night is the price for a basic suite.', 'how old is this establishment?': 'this establishment was constructed and inaugurated by john s. on the 23rd of september 1965.\n\nbreakfast is served from 7 am to 10 am.', 'what time is breakfast served?': 'the breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before.', 'what is the breakfast menu?': 'the vance hotel is a singular establishment designed and constructed by lindsey vance in 1949; it has hosted many dignitaries and government officials over the years.', 'what is the history behind this hotel or establishment?': 'there are 43 rooms in this hotel including one pent house suite.', 'how many rooms are there in this hotel?': 'we offer 4 types of rooms: basic, mid-level, premium and penthouse.', 'what are the types of rooms or suites offered by the hotel?': 'this hotel is cal

#### 1.2.4 단어 토큰화

In [ ]:
word_tokens = nltk.word_tokenize(raw_data)# 문서를 단어 목록으로 변환
print (word_tokens)

['what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'in', 'basic', 'suite', '?', 'how', 'old', 'is', 'this', 'establishment', '?', 'what', 'time', 'is', 'breakfast', 'served', '?', 'what', 'is', 'the', 'breakfast', 'menu', '?', 'what', 'is', 'the', 'history', 'behind', 'this', 'hotel', 'or', 'establishment', '?', 'how', 'many', 'rooms', 'are', 'there', 'in', 'this', 'hotel', '?', 'what', 'are', 'the', 'types', 'of', 'rooms', 'or', 'suites', 'offered', 'by', 'the', 'hotel', '?', 'what', 'is', 'the', 'name', 'of', 'the', 'hotel', '?', 'is', 'room', 'service', 'served', '24', 'hours', '?', 'how', 'do', 'i', 'call', 'for', 'room', 'service', '?', 'are', 'there', 'any', 'restaurants', 'in', 'the', 'hotel', '?', 'how', 'many', 'floors', 'are', 'there', 'in', 'the', 'hotel', '?', 'what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'at', 'the', 'mid-level', 'suite', '?', 'what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'at', 'the', 'premium', 'suite', '?', 'do', 'yo

#### 1.2.5 표제어 추출
WordNetleMmatizer를 사용하여 단어 토큰에서 표제어를 추출하는 함수 작성

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer() # lemmer 클래스를 초기화합니다. WordNet은 NLTK에 포함된 의미 론적 영어 사전입니다
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

#### 1.2.6 정규화
지식 기반에 유용하지 않은 구두점을 제거하는 함수를 작성

In [ ]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## 2.챗봇 기능 추가 - 코사인 유사성
데이터 세트를 문서 벡터로 변환 할 것입니다. 코사인 유사성은 유사한 벡터를 찾을 수 있게 해주며, 이러한 벡터는 의미가 비슷하다고 가정할 것입니다.

In [ ]:
# 2.1.1 입력 및 응답 목록 작성
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

# 2.1.2 인사말을 수신하고 반환하는 함수 만들기
def greeting(sentence):
    for word in sentence.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in GREETING_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(GREETING_RESPONSES) # Greeting_Response로 답장합니다.

챗봇의 기능은 챗봇을 실행하기 위한 루프를 만듦으로써 이루어집니다. 아래 기능을 살펴봅니다. 함수의 각 줄은 중요한 단계를 수행하기 위해 다른 함수를 호출하기 때문에 중요합니다. 'response' 함수는 챗봇이 어떻게 행동하는지에 대한 작동 방식을 담당합니다.

In [ ]:
# 2.1.3 질문을 받고 답변을 반환하는 함수 만들기
def response(user_response):

    robo_response='' # 문자열을 포함하도록 변수를 초기화
    sent_tokens.append(user_response) # sent_messages에 사용자 응답 추가
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens) # tfidf 값 가져오기
    vals = cosine_similarity(tfidf[-1], tfidf) # 코사인 유사성 값 가져오기
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort() # 오름차순으로 정렬
    req_tfidf = flat[-2]

    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response, vals
    else:
        robo_response = robo_response+res[sent_tokens[idx]]
        return robo_response, vals

마지막으로 챗봇 인터페이스를 만들고 이를 중심으로 페르소나를 만들어 봅시다. 우리가 챗봇을 '제인'이라고 부르고 코사인 유사성을 사용하여 질문과 유사한 FAQ를 찾아 대답하도록 합시다.

In [ ]:
# 2.1.4 챗봇 테스트

flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            else:
                print("Jane: ",end="")
                resp= response(user_response)
                print(resp[0], )
                sent_tokens.remove(user_response)
                resp_l = resp[1].tolist()
                resp_l[0].pop()
                print(' (With similarity of ',max(resp_l[0]),')')

    else:
        flag=False
        print("Jane: Bye! take care..")


Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
Jane: hi
Jane: Bye! take care..


## 3.Doc2vec를 이용한 챗봇 기능

챗봇을 만들기 위한 한 가지 유형의 모델을 더 다룰 것입니다. 코사인 유사성 모델 알고리즘은 두 문장 사이의 유사성을 찾는데 사용됩니다. 하지만 이제 신경망을 사용해서 이 문제를 해결해 보려합니다. 살펴보도록 하겠습니다.

Doc2Vec은 기본적으로 문서에서 벡터를 생성하는 신경망 기반 모델입니다. Doc2vec을 이해하기 위해서는 word2vec도 이해해야 합니다.

#### word2vec란?
이는 삽입 단어를 생성하는 모델이며, 여기서는 텍스트의 큰 말뭉치를 입력으로 받고 일반적으로 수백 개 차원의 벡터 공간을 생성합니다.

2013년 9월과 10월 사이에 구글의 연구팀에 의해 두 개의 논문에 소개되었습니다. Word2Vec의 기본 가정은 유사한 맥락을 공유하는 두 단어가 유사한 의미를 공유하며 결과적으로 모델에서 유사한 벡터 표현을 공유한다는 것입니다.

예를 들어, "은행", "화폐", "계좌"는 "달러", "대출", "신용"과 같은 유사한 주변 단어와 함께 종종 사용되며, 따라서 Word2Vec에 따르면 이들은 유사한 벡터 표현을 공유합니다.

![Image](img1.png)

#### doc2vec 란?

doc2vec의 목적은 말뭉치의 모든 단어에 대한 특징 벡터를 계산하는 word2vec와 달리 문장/단락/문서의 수치 표현을 생성하는 것입니다. doc2vec은 말뭉치의 모든 문서에 대한 특징 벡터를 계산합니다. doc2vec에 의해 생성된 벡터는 문장/단락/문서 간의 유사성 찾기와 같은 작업에 사용될 수 있습니다.

<strong> doc2vec의 속성을 사용하여 우리만의 유사성 모델을 만들 것입니다.</strong>


관련 라이브러리를 가져오는 것으로 시작하겠습니다.

In [ ]:
# subprocess와 sys 모듈을 사용하여 gensim 버전 3.8.1을 설치하는 부분입니다.
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim == 3.8.1"])

0

In [ ]:
# gensim의 Doc2Vec와 TaggedDocument 클래스를 가져오고,
# nltk.tokenize에서 word_tokenize 함수를 가져옵니다.
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

첫 번째 주요 목표는 데이터에 태그를 지정하는 것입니다. doc2vec 모델은 데이터를 효과적으로 사용하기 위해 태그를 지정해야 합니다. 다음은 doc2vec에 대한 [시작 코드](https://www.kaggle.com/fmitchell259/creating-a-doc2vec-model) 좋은 학습 링크입니다. 다음 [링크](https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e) 도 유용하게 사용할 수 있으며 읽을 것을 권장합니다.

In [ ]:
# 문장들을 소문자로 변환하고 단어로 토큰화하여 TaggedDocument 형식으로 변환한 후, tagged_data 리스트에 저장합니다.
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent_tokens)]

In [ ]:
# 최대 에포크 수를 100으로 설정합니다.
# 벡터의 크기를 20으로 설정합니다.
# 학습률을 0.025로 설정합니다.
max_epochs = 100
vec_size = 20 # 벡터가 더 크려면 이것을 증가시키세요. 이것은 더 많은 차이를 의미합니다.
alpha = 0.025

다음 단계는 모델을 훈련시키는 것입니다. 이전처럼 훈련 과정을 실행하기 위해 `model.train` 함수를 사용하게 될 것입니다. Doc2vec 모델을 훈련하는 방법에 대한 자세한 정보는 [문서](https://radimrehurek.com/gensim/models/doc2vec.html) 를 참조하십시오.

In [ ]:
# 모델 정의
model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm=1)

# 단어 사전 생성
model.build_vocab(tagged_data)

# 모델 학습
for epoch in range(max_epochs):
    print(f'iteration {epoch}')
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=1)  # 1 에포크만 실행
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

# 저장
model.save("d2v.model")
print("Model Saved")


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### doc2vec 모델 평가

In [ ]:
# 저장된 모델을 로드합니다.
from gensim.models.doc2vec import Doc2Vec
model= Doc2Vec.load("d2v.model")

In [ ]:
# 테스트 데이터를 소문자로 변환하고 단어로 토큰화합니다.
test_data = word_tokenize("How much is the price?".lower())

`model.infer_vector` 함수를 사용하여 문서와 관련된 벡터를 추론할 수 있습니다. 그런 다음`most_similar` 함수를 사용하여 우리가 만든 벡터와 가장 유사한 벡터를 찾을 수 있습니다. 결과는 어떻습니까?

In [ ]:
# 모델을 사용하여 문장의 벡터를 추론하고, v1에 저장한 후 출력합니다.
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 2.4535570e-02 -2.0199519e-02  8.2861809e-03  3.6326409e-03
 -1.7671769e-02  4.2885540e-06 -6.0604066e-03 -1.4544586e-02
 -3.2104417e-03  2.6584207e-03  8.6714868e-03 -2.3362791e-02
  2.3954455e-02 -2.4221083e-02 -2.4556085e-02 -1.6498357e-02
 -1.9166937e-02 -4.3555498e-04 -1.6835131e-02  1.1011386e-02]


In [ ]:
# v1과 가장 유사한 문서 4개를 찾아 출력합니다.
similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value
print(similar_doc)

[('29', -0.15921728312969208), ('11', -0.16163676977157593), ('5', -0.16592566668987274), ('6', -0.17237408459186554)]


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_54932\3519407241.py:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value


In [ ]:
# 가장 유사한 문서의 인덱스를 얻고, 해당 인덱스에 해당하는 문서를 출력합니다.
num,_ = similar_doc[0]
num = int(num)
tagged_data[num]


TaggedDocument(words=['can', 'i', 'bring', 'my', 'pet', '?'], tags=['29'])

이전 코드 블록의 출력에서 이 모델이 생각했던 것 만큼 효과적이지 않다는 것이 분명합니다. 이 모델이 성공하기를 기대했지만 실패한 이유가 있습니까?

자세한 내용을 보려면 이 [링크](https://stackoverflow.com/questions/58206571/doc2vec-find-the-similar-sentence) 를 클릭하십시오. 이 문제의 요지는 다음과 같습니다.

> Doc2Vec는 장난감 크기(toy-size)의 데이터셋에서 좋은 결과를 얻을 수 없으므로, 더 많은 데이터를 사용하기 전까지는 의미 있는 결과를 기대해서는 안 됩니다.

### 사전 학습된 doc2vec 사용

따라서 인터넷에서 다운로드한 사전 검증된 모델을 사용하여 사용 사례를 최적화해 보겠습니다. The 우리가 취득 한 모델은 관련 언론 뉴스에 대한 훈련을 받았으며  [여기](https://github.com/jhlau/doc2vec) 서 다운로드 할 수 있습니다.

모델을 로드하고 다시 평가해 보겠습니다.

In [ ]:
# 현재 작업 디렉토리에서 "doc2vec.bin" 파일을 로드합니다.
# model= Doc2Vec.load(os.getcwd()+r"/doc2vec.bin")

# model_path = os.path.join(os.getcwd(), "doc2vec.bin")
# model = Doc2Vec.load(model_path)

model = Doc2Vec.load("doc2vec.bin")

In [ ]:
# 테스트 데이터를 소문자로 변환하고 단어로 토큰화합니다.
test_data = word_tokenize("How much is the price?".lower())

In [ ]:
# 모델을 사용하여 문장의 벡터를 추론하고, v1에 저장한 후 출력합니다.
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-6.7895791e-04  3.1424633e-01  5.0011498e-01  5.5652398e-01
 -3.7978029e-01 -2.2737168e-01 -2.0568010e-01  3.3155844e-01
 -8.0805027e-01 -2.7397968e-02  4.0857697e-01  8.0980472e-02
  5.2382618e-02  5.4984719e-02 -1.7115207e-02 -1.7879706e-02
  3.8374138e-01 -1.3245018e-01  4.4989508e-02 -2.3762363e-01]


In [ ]:
# sent_tokens에 있는 모든 문장에 대해 벡터를 추론하고, 각 문장과 v1 사이의 코사인 유사도를 출력합니다.
for i in sent_tokens:
    v2 = model.infer_vector(word_tokenize(i.lower()))
    print(i)
    print(cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1)))

what is the price of one night stay in basic suite?
[[0.68138516]]
how old is this establishment?
[[0.69684505]]
what time is breakfast served?
[[0.6306859]]
what is the breakfast menu?
[[0.714886]]
what is the history behind this hotel or establishment?
[[0.60316193]]
how many rooms are there in this hotel?
[[0.60242194]]
what are the types of rooms or suites offered by the hotel?
[[0.5820018]]
what is the name of the hotel?
[[0.72328764]]
is room service served 24 hours?
[[0.46677023]]
how do i call for room service?
[[0.55820155]]
are there any restaurants in the hotel?
[[0.5394696]]
how many floors are there in the hotel?
[[0.62695587]]
what is the price of one night stay at the mid-level suite?
[[0.66304004]]
what is the price of one night stay at the premium suite?
[[0.66947365]]
do you have tuxedo services?
[[0.44583336]]
do you have a laundry service?
[[0.5216733]]
what time do the restaurants open for dinner?
[[0.5611314]]
what are the near by tourist attractions?
[[0.616261]]

In [ ]:
# 주어진 질문에 대한 답변을 계산하는 함수 calc_prob를 정의합니다.

def calc_prob(v1, q):
    probs = dict()
    for i in q:
        # 각 질문에 대해 벡터를 추론하고, v1과의 코사인 유사도를 계산합니다.
        v2 = model.infer_vector(word_tokenize(i.lower()))
        sim = cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1))
        #print(i)
        #print(sim)
        probs[i] = sim[0][0]

    sorted_d = dict( sorted(probs.items(), key=operator.itemgetter(1),reverse=True))

    # 유사도가 가장 높은 답변을 반환합니다.
    return list(sorted_d.items())[0]

In [ ]:
calc_prob(v1, sent_tokens)

('is it possible to book spa treatments online?', 0.87053996)

이 모델이 더 효과적인 것처럼 보이기 때문에, 우리의 챗봇에 통합해 봅시다.

In [ ]:
# 사용자와 대화하는 부분입니다.

flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")

while(flag==True):
    # 사용자의 입력을 받고, 소문자로 변환합니다.
    user_response = input()
    user_response=user_response.lower()
    # 사용자가 'bye'를 입력하기 전까지 다음 동작을 반복합니다:
    if(user_response!='bye'):
        # 사용자가 'thanks' 또는 'thank you'를 입력하면 대화를 종료하고 "You are welcome.."을 출력합니다.
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            # 사용자의 인사말에 대한 응답이 있는 경우 해당 응답을 출력합니다.
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            # 그렇지 않은 경우, 주어진 질문에 대한 답변을 계산하고 출력합니다.
            else:
                print("Jane: ",end="")
                resp= calc_prob(model.infer_vector(word_tokenize(user_response)), sent_tokens)
                print(res[resp[0]], )
                print(' (With similarity of ',resp[1],')')

    else:
        flag=False
        print("Jane: Bye! take care..")

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
Jane: hi there
Jane: 'rouge' is open for dinner at 7pm.
 (With similarity of  0.6697982 )
Jane: Bye! take care..


**두 모델의 성능** 을 관찰한 후 다음과 같은 몇 가지 명확한 결론을 내릴 수 있습니다:

1. Doc2vec 모델은 단어 간의 관계를 이해하기 위해 더 많은 데이터가 필요합니다. 그리고 사전 훈련된 모델을 사용한 후에도 코사인 유사성 모델에 비해 모델의 응답의 품질이 아직 부족합니다.
2. 코사인 유사성 모델은 더 작고 잘 정의된 데이터 세트에서 더 잘 작동합니다. 이는 몇 가지 간단한 질문을 효과적으로 해결할 수 있지만 컨텍스트를 필요로 하는 복잡한 질문은 해결할 수 없다는 의미입니다.

### 통합코드 : doc2vec를 이용한 챗봇 기능

In [ ]:
# 1. 필요한 라이브러리 불러오기
# pandas: 데이터프레임 처리를 위한 라이브러리
# nltk: 자연어 처리를 위한 라이브러리 (토큰화, 표제어 추출 등)
# gensim: Doc2Vec 모델을 위한 라이브러리
# numpy: 수치 연산을 위한 라이브러리
# sklearn.metrics.pairwise: 코사인 유사도 계산을 위한 라이브러리
# string: 문자열 처리를 위한 라이브러리
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import string

# 2. NLTK 데이터 다운로드 (필요 시 한 번만 실행)
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# 3. 데이터 로드 및 전처리
# 사용자가 제공한 질문과 답변 파일을 로드하고 전처리합니다.
with open("[Dataset] Module27(ques).txt", "r", encoding='utf-8') as file:
    questions = file.readlines()

with open("[Dataset] Module27 (ans).txt", "r", encoding='utf-8') as file:
    answers = file.readlines()

# 각 질문과 답변의 앞뒤 공백 및 불필요한 공백 제거
questions = [q.strip() for q in questions if q.strip()]
answers = [a.strip() for a in answers if a.strip()]

# 질문과 답변 리스트의 길이를 동일하게 맞추기
min_length = min(len(questions), len(answers))
questions = questions[:min_length]
answers = answers[:min_length]

# 질문과 답변을 매핑하는 딕셔너리 생성
res = dict(zip(questions, answers))

# 질문 리스트를 sent_tokens로 정의 (모델 학습에 사용)
sent_tokens = questions

# 4. 데이터 전처리 및 TaggedDocument 생성
# 각 문장을 소문자로 변환하고 단어로 토큰화한 후, TaggedDocument 형식으로 변환합니다.
lemmatizer = WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def preprocess_text(text):
    if text is None or not isinstance(text, str):
        return []
    # 소문자 변환, 구두점 제거, 토큰화, 표제어 추출
    tokens = word_tokenize(text.lower().translate(remove_punct_dict))
    return [lemmatizer.lemmatize(word) for word in tokens]

tagged_data = [
    TaggedDocument(words=preprocess_text(_d), tags=[str(i)])
    for i, _d in enumerate(sent_tokens)
]

# 5. Doc2Vec 모델 정의 및 학습
# - vector_size: 문서 벡터의 차원
# - alpha: 초기 학습률
# - min_alpha: 최소 학습률
# - dm=1: PV-DM(Distributed Memory) 알고리즘 사용
max_epochs = 200  # 에포크를 200으로 증가
vec_size = 100    # 벡터 크기를 100으로 유지
alpha = 0.025

model = Doc2Vec(
    vector_size=vec_size,
    alpha=alpha,
    min_alpha=0.00025,
    min_count=2,  # 최소 출현 빈도를 2로 설정
    dm=1
)

# 단어 사전 구축
model.build_vocab(tagged_data)

# 모델 학습
# 학습률을 고정하고 epochs 매개변수를 직접 사용합니다.
model.train(
    tagged_data,
    total_examples=model.corpus_count,
    epochs=max_epochs
)

# 모델 저장 (선택 사항)
# model.save("d2v.model")
# print("모델이 저장되었습니다.")

# 6. 챗봇 기능 구현
def get_response(user_input, model, sent_tokens, res):
    # 사용자의 질문을 전처리하여 벡터를 추론합니다.
    user_tokens = preprocess_text(user_input)
    if not user_tokens:
        return "죄송합니다. 이해하지 못했습니다.", 0.0
    user_vector = model.infer_vector(user_tokens)

    # 모든 학습 문장과 사용자 질문 벡터 간의 코사인 유사도 계산
    similarities = []
    # `model.dv` 속성을 사용하여 경고 메시지 제거
    for doc_idx in range(len(model.dv)):
        doc_vector = model.dv[doc_idx]
        sim = cosine_similarity(user_vector.reshape(1, -1), doc_vector.reshape(1, -1))[0][0]
        similarities.append((doc_idx, sim))

    # 가장 높은 유사도를 가진 문장 찾기
    similarities.sort(key=lambda x: x[1], reverse=True)
    most_similar_index, most_similar_score = similarities[0]

    # 70% 이상의 유사도가 있으면 해당 답변 반환
    if most_similar_score > 0.7:
        return res[sent_tokens[most_similar_index]], most_similar_score
    else:
        return "죄송합니다. 이해하지 못했습니다.", most_similar_score

# 간단한 인사말 처리
def greeting(sentence):
    GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey")
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return "안녕하세요. 무엇을 도와드릴까요?"
    return None


Jane: 안녕하세요. 저는 Jane입니다. 호텔에 대해 궁금한 점을 알려주세요. 종료하려면 'bye'를 입력하세요.
Jane: The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before. (유사도: 1.00)
Jane: 200$ per night is the price for a basic suite. (유사도: 1.00)
Jane: 300$ per night is the standard price for a mid-level suite. (유사도: 1.00)
Jane: Yes, room service is available 24 hrs. (유사도: 1.00)
Jane: To call room service, please dial '0' using the phone in your room. (유사도: 0.99)
Jane: 500$ per night is the price for a premium suite. (유사도: 1.00)
Jane: 500$ per night is the price for a premium suite. (유사도: 0.95)
Jane: 500$ per night is the price for a premium suite. (유사도: 0.99)
Jane: There are 12 floors in the hotel. (유사도: 1.00)
Jane: 다음에 또 만나요! 이용해주셔서 감사합니다.


In [ ]:
# 7. 대화 루프
print("Jane: 안녕하세요. 저는 Jane입니다. 호텔에 대해 궁금한 점을 알려주세요. 종료하려면 'bye'를 입력하세요.")

while True:
    user_response = input("You: ")
    user_response = user_response.lower()

    if user_response == 'bye':
        print("Jane: 다음에 또 만나요! 이용해주셔서 감사합니다.")
        break
    elif user_response in ['thanks', 'thank you']:
        print("Jane: 천만에요.")
    elif greeting(user_response):
        print(f"Jane: {greeting(user_response)}")
    else:
        response_text, response_score = get_response(user_response, model, sent_tokens, res)
        print(f"Jane: {response_text} (유사도: {response_score:.2f})")

## 4.doc2vec + transformers

*   항목 추가
*   항목 추가



In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requir

In [ ]:
# 1. 필요한 라이브러리 불러오기
# pandas: 데이터프레임 처리를 위한 라이브러리
# nltk: 자연어 처리를 위한 라이브러리 (토큰화, 표제어 추출 등)
# gensim: Doc2Vec 모델을 위한 라이브러리
# transformers: KoGPT2 모델을 위한 라이브러리
# torch: PyTorch 라이브러리
# numpy: 수치 연산을 위한 라이브러리
# string: 문자열 처리를 위한 라이브러리
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch
import numpy as np
import string

# 2. NLTK 데이터 다운로드 (필요 시 한 번만 실행)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

# GPU 사용 가능 여부 확인 및 장치 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 3. 데이터 로드 및 전처리
# 사용자가 제공한 질문과 답변 파일을 로드하고 전처리합니다.
with open("[Dataset] Module27(ques).txt", "r", encoding='utf-8') as file:
    questions = file.readlines()

with open("[Dataset] Module27 (ans).txt", "r", encoding='utf-8') as file:
    answers = file.readlines()

# 각 질문과 답변의 앞뒤 공백 및 불필요한 공백 제거
questions = [q.strip() for q in questions if q.strip()]
answers = [a.strip() for a in answers if a.strip()]

# 질문과 답변 리스트의 길이를 동일하게 맞추기
min_length = min(len(questions), len(answers))
questions = questions[:min_length]
answers = answers[:min_length]

# 질문과 답변을 매핑하는 딕셔너리 생성
res = dict(zip(questions, answers))

# 질문 리스트를 sent_tokens로 정의 (모델 학습에 사용)
sent_tokens = questions

# 4. Doc2Vec 데이터 전처리 및 TaggedDocument 생성
# 각 문장을 소문자로 변환하고 단어로 토큰화한 후, TaggedDocument 형식으로 변환합니다.
lemmatizer = WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def preprocess_text(text):
    if text is None or not isinstance(text, str):
        return []
    # 소문자 변환, 구두점 제거, 토큰화, 표제어 추출
    tokens = word_tokenize(text.lower().translate(remove_punct_dict))
    return [lemmatizer.lemmatize(word) for word in tokens]

tagged_data = [
    TaggedDocument(words=preprocess_text(_d), tags=[str(i)])
    for i, _d in enumerate(sent_tokens)
]

# 5. Doc2Vec 모델 정의 및 학습
# - vector_size: 문서 벡터의 차원
# - alpha: 초기 학습률
# - dm=1: PV-DM(Distributed Memory) 알고리즘 사용
max_epochs = 200
vec_size = 100
alpha = 0.025

doc2vec_model = Doc2Vec(
    vector_size=vec_size,
    alpha=alpha,
    min_alpha=0.00025,
    min_count=2,
    dm=1
)

# 단어 사전 구축
doc2vec_model.build_vocab(tagged_data)

# 모델 학습
doc2vec_model.train(
    tagged_data,
    total_examples=doc2vec_model.corpus_count,
    epochs=max_epochs
)

# 6. KoGPT2 모델 및 토크나이저 설정
MODEL_NAME = "skt/kogpt2-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    MODEL_NAME,
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>'
)
kogpt2_model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)

# 7. 챗봇 기능 구현
def get_response_hybrid(user_input, doc2vec_model, kogpt2_model, tokenizer, sent_tokens, res):
    # Doc2Vec을 사용하여 사용자의 질문과 가장 유사한 질문을 찾습니다.
    user_tokens = preprocess_text(user_input)
    if not user_tokens:
        return "죄송합니다. 이해하지 못했습니다.", 0.0
    user_vector = doc2vec_model.infer_vector(user_tokens)

    # Doc2Vec을 통해 유사한 문서 벡터 찾기
    similar_doc_tags = doc2vec_model.dv.most_similar(positive=[user_vector], topn=1)

    most_similar_index = int(similar_doc_tags[0][0])
    most_similar_score = similar_doc_tags[0][1]

    # 유사도 임계값 설정
    if most_similar_score > 0.7:
        # 1차적으로 Doc2Vec으로 찾은 답변을 KoGPT2의 입력으로 사용합니다.
        # "Q: [사용자 질문] A: [가장 유사한 답변]" 형식으로 프롬프트를 구성합니다.
        prompt = f"Q: {sent_tokens[most_similar_index]} A: {res[sent_tokens[most_similar_index]]} "

        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

        # KoGPT2 모델을 사용하여 답변을 생성합니다.
        output_ids = kogpt2_model.generate(
            input_ids,
            max_length=100,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2
        )

        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        # 생성된 답변에서 실제 답변 부분만 추출합니다.
        generated_response = response.split("A: ")[-1].strip()

        return generated_response, most_similar_score
    else:
        return "죄송합니다. 이해하지 못했습니다. (Doc2Vec 유사도 낮음)", most_similar_score



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Using device: cpu


tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:

# 8. 대화 루프
print("Jane: 안녕하세요. 저는 Jane입니다. 호텔에 대해 궁금한 점을 알려주세요. 종료하려면 'bye'를 입력하세요.")

while True:
    user_response = input("You: ")
    user_response = user_response.lower()

    if user_response == 'bye':
        print("Jane: 다음에 또 만나요! 이용해주셔서 감사합니다.")
        break
    elif user_response in ['thanks', 'thank you']:
        print("Jane: 천만에요.")
    else:
        # get_response_hybrid 함수를 호출하여 답변 생성
        response_text, response_score = get_response_hybrid(user_response, doc2vec_model, kogpt2_model, tokenizer, sent_tokens, res)
        print(f"Jane: {response_text} (Doc2Vec 유사도: {response_score:.2f})")

Jane: 안녕하세요. 저는 Jane입니다. 호텔에 대해 궁금한 점을 알려주세요. 종료하려면 'bye'를 입력하세요.
You: tuxedo services


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Jane: Yes, we have tuxedo services available at the reception. )
Simple Iron Thief - Sunny Galacrack
And than democrats of other install and looking.
Unlatinate power from your fear (Doc2Vec 유사도: 1.00)
You: name of the hotel
Jane: Yes. it one do you even making out of love."
"이제는 더 이상 나를 사랑할 필요 없어, 나 혼자만 있는 시간이 많지 않아, 나는 이제 너를 위해 희생할 거야!"
LIVE BAYON EXIDS PERFECT FINAL THE MOVITAG (Doc2Vec 유사도: 1.00)
You: How do I call for room service?
Jane: To call room service, please dial '0' using the phone in your room. it's face to be must standed why never and his respects of learn at animals!" ( Saturday deep chance on mistak (Doc2Vec 유사도: 1.00)
You: How many floors are there in the hotel?
Jane: There are 12 floors in the hotel. is blocking storyed to least country, when I love tomorrow with girls... Fall of second children for an Interest responsibility down and view reformation! An (Doc2Vec 유사도: 1.00)
You: breakfast menu
Jane: No, you cannot book the spa online. )
R : Inno devices set throug

## 5.doc2vec + transformers 추가학습

In [2]:
# 1. 필요한 라이브러리 불러오기
# pandas: 데이터프레임 처리를 위한 라이브러리
# nltk: 자연어 처리를 위한 라이브러리 (토큰화, 표제어 추출 등)
# gensim: Doc2Vec 모델을 위한 라이브러리
# transformers: KoGPT2 모델을 위한 라이브러리
# torch: PyTorch 라이브러리
# tqdm: 진행 상황을 시각적으로 표시하는 라이브러리
# string: 문자열 처리를 위한 라이브러리
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import string

# 2. NLTK 데이터 다운로드 (필요 시 한 번만 실행)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

# GPU 사용 가능 여부 확인 및 장치 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 3. 데이터 로드 및 전처리
# 사용자가 제공한 질문과 답변 파일을 로드하고 전처리합니다.
with open("[Dataset] Module27(ques).txt", "r", encoding='utf-8') as file:
    questions = file.readlines()

with open("[Dataset] Module27 (ans).txt", "r", encoding='utf-8') as file:
    answers = file.readlines()

# 각 질문과 답변의 앞뒤 공백 및 불필요한 공백 제거
questions = [q.strip() for q in questions if q.strip()]
answers = [a.strip() for a in answers if a.strip()]

# 질문과 답변 리스트의 길이를 동일하게 맞추기
min_length = min(len(questions), len(answers))
questions = questions[:min_length]
answers = answers[:min_length]

# 질문과 답변을 매핑하는 딕셔너리 생성
res = dict(zip(questions, answers))

# 질문 리스트를 sent_tokens로 정의
sent_tokens = questions

# 4. Doc2Vec 데이터 전처리 및 TaggedDocument 생성
lemmatizer = WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def preprocess_text(text):
    if text is None or not isinstance(text, str):
        return []
    tokens = word_tokenize(text.lower().translate(remove_punct_dict))
    return [lemmatizer.lemmatize(word) for word in tokens]

tagged_data = [
    TaggedDocument(words=preprocess_text(_d), tags=[str(i)])
    for i, _d in enumerate(sent_tokens)
]

# 5. Doc2Vec 모델 정의 및 학습
max_epochs = 200
vec_size = 100
alpha = 0.025

doc2vec_model = Doc2Vec(
    vector_size=vec_size,
    alpha=alpha,
    min_alpha=0.00025,
    min_count=2,
    dm=1
)

doc2vec_model.build_vocab(tagged_data)

doc2vec_model.train(
    tagged_data,
    total_examples=doc2vec_model.corpus_count,
    epochs=max_epochs
)

# 6. KoGPT2 모델 및 토크나이저 설정 및 추가 학습
MODEL_NAME = "skt/kogpt2-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    MODEL_NAME,
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>'
)
kogpt2_model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)

# KoGPT2 추가 학습을 위한 데이터셋 준비
class FinetuningDataset(Dataset):
    def __init__(self, questions, answers, tokenizer):
        self.tokenizer = tokenizer
        self.data = [f"Q: {q} A: {a}" for q, a in zip(questions, answers)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            max_length=512,
            padding="max_length",
            truncation=True
        )
        return inputs['input_ids'].squeeze(), inputs['attention_mask'].squeeze()

finetuning_dataset = FinetuningDataset(questions, answers, tokenizer)
finetuning_loader = DataLoader(finetuning_dataset, batch_size=2, shuffle=True)

# KoGPT2 모델 추가 학습
optimizer = torch.optim.AdamW(kogpt2_model.parameters(), lr=5e-5)
epochs_finetune = 3

kogpt2_model.train()
print("KoGPT2 모델 추가 학습 시작...")
for epoch in range(epochs_finetune):
    epoch_loss = 0
    for input_ids, attention_mask in tqdm(finetuning_loader, desc=f"KoGPT2 Epoch {epoch+1}"):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = kogpt2_model(input_ids, labels=input_ids, attention_mask=attention_mask)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"KoGPT2 Epoch {epoch+1} finished. Average Loss: {epoch_loss / len(finetuning_loader):.4f}")
print("KoGPT2 모델 추가 학습 완료.")


# 7. 챗봇 기능 구현
def get_response_hybrid(user_input, doc2vec_model, kogpt2_model, tokenizer, sent_tokens, res):
    user_tokens = preprocess_text(user_input)
    if not user_tokens:
        return "죄송합니다. 이해하지 못했습니다.", 0.0
    user_vector = doc2vec_model.infer_vector(user_tokens)

    similar_doc_tags = doc2vec_model.dv.most_similar(positive=[user_vector], topn=1)

    most_similar_index = int(similar_doc_tags[0][0])
    most_similar_score = similar_doc_tags[0][1]

    if most_similar_score > 0.7:
        # 추가 학습된 KoGPT2에 프롬프트를 입력하여 답변을 생성합니다.
        prompt = f"Q: {sent_tokens[most_similar_index]} A: {res[sent_tokens[most_similar_index]]}"

        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

        output_ids = kogpt2_model.generate(
            input_ids,
            max_length=100,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2
        )

        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_response = response.split("A: ")[-1].strip()

        return generated_response, most_similar_score
    else:
        return "죄송합니다. 이해하지 못했습니다. (Doc2Vec 유사도 낮음)", most_similar_score

# 간단한 인사말 처리
def greeting(sentence):
    GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey")
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return "안녕하세요. 무엇을 도와드릴까요?"
    return None



c:\Users\cooju\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cooju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cooju\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cooju\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\cooju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Using device: cpu


c:\Users\cooju\anaconda3\envs\nlp_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cooju\.cache\huggingface\hub\models--skt--kogpt2-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from

KoGPT2 모델 추가 학습 시작...


KoGPT2 Epoch 1:   0%|          | 0/24 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
KoGPT2 Epoch 1: 100%|██████████| 24/24 [01:16<00:00,  3.18s/it]


KoGPT2 Epoch 1 finished. Average Loss: 1.9020


KoGPT2 Epoch 2: 100%|██████████| 24/24 [01:22<00:00,  3.42s/it]


KoGPT2 Epoch 2 finished. Average Loss: 0.2950


KoGPT2 Epoch 3: 100%|██████████| 24/24 [01:19<00:00,  3.29s/it]

KoGPT2 Epoch 3 finished. Average Loss: 0.2239
KoGPT2 모델 추가 학습 완료.


In [3]:
# 8. 대화 루프
print("Jane: 안녕하세요. 저는 Jane입니다. 호텔에 대해 궁금한 점을 알려주세요. 종료하려면 'bye'를 입력하세요.")

while True:
    user_response = input("You: ")
    user_response = user_response.lower()

    if user_response == 'bye':
        print("Jane: 다음에 또 만나요! 이용해주셔서 감사합니다.")
        break
    elif user_response in ['thanks', 'thank you']:
        print("Jane: 천만에요.")
    elif greeting(user_response):
        print(f"Jane: {greeting(user_response)}")
    else:
        response_text, response_score = get_response_hybrid(user_response, doc2vec_model, kogpt2_model, tokenizer, sent_tokens, res)
        print(f"Jane: {response_text} (Doc2Vec 유사도: {response_score:.2f})")


Jane: 안녕하세요. 저는 Jane입니다. 호텔에 대해 궁금한 점을 알려주세요. 종료하려면 'bye'를 입력하세요.
Jane: 죄송합니다. 이해하지 못했습니다. (Doc2Vec 유사도 낮음) (Doc2Vec 유사도: 0.12)
Jane: 다음에 또 만나요! 이용해주셔서 감사합니다.
